## CURRENT NOTES

(02/24/20):

~ I've added the reproduction function

~ It can be used in a .apply() manner

(02/12/20):

~ There are now functions for gene transfer and two different ways of displaying data

~ Still need work on bacteria reproduction function. A few decisions need to be made (likely as a group)


(02/10/20):

~ There are no functions yet

~ Let's work this week to implement the determinisitic model sent to us

---

- This code establishes the bacteria class to be used in the agent-based model

- The final model does not have to be done in this notebook

- This should just be the building ground for the individuals and their functions in the ABM

In [1]:
import networkx as nx
import pandas as pd
from matplotlib import pyplot as plt
import datetime
import numpy as np 
import math
import pylab as pl
import math
import random as rd 
import copy

In [2]:
##initializing the bacteria class

class bacteria:
    
    def __init__(self,state,tax,gene_pres,gene,mge_pres,mge,biome):
        
        self.state = state #{0:susceptible, 1:resistant, 2:acquired resistant}
        self.tax = tax
        self.gene_pres = gene_pres
        self.gene = gene
        self.mge_pres = mge_pres
        self.mge = mge
        self.biome = biome
        
    def display(self):
        #will display all features of bacteria
        print("State: ",self.state)
        print("Taxonomy: ",self.tax)
        print("ARG Presence: ",self.gene_pres)
        print("ARG: ",self.gene)
        print("MGE Presence: ",self.mge_pres)
        print("MGE: ",self.mge)
        print("Biome: ",self.biome)
        print("")
        
    def build_series(self):
        row = pd.Series([self.state,self.tax,self.gene_pres,self.gene,self.mge_pres,self.mge,self.biome],
                        index=["state","taxonomy","gene_pres","gene","mge_pres","mge","biome"])
        return row

In [3]:
##This function changes the state to acuired resistance
##Meant to be used in a .apply or map() manner
def gene_transferred(x):
    x.state = 2
    return x

In [4]:
##colony is a list of bacteria class objects
##This function reassigns the states of susceptible bacteria based on the horizontal gene transfer rates

##WHEN DONE TESTING BE SURE TO REMOVE PRINT() LINES

def horizontal_gene_transfer(colony,acq_trans_rate=0.0001,res_trans_rate=0.0001):
    
    ##Number of newly resistant bacteria from acquired resistance bacteria
    new_acq = acq_trans_rate*len([x for x in colony if x.state == 2])
    
    ##Number of newly resistant bacteria from originally resistant bacteria
    new_res = res_trans_rate*len([x for x in colony if x.state == 1])
    
    ##randomly choose which bacteria will be new resistant bacteria
    print(new_acq+new_res)
    changing = rd.sample([x for x in colony if x.state == 0],round(new_acq+new_res))
    print(changing)
    list(map(gene_transferred,[x for x in changing if x.state == 0])) #This will globally change state; don't worry about it being a list
    
    
    return #Nothing to return since changes were global
    

In [11]:
def group_df(bac_list):
    ##bac_list must be a list of bacteria objects
    ##this can be a full colony or any specified subgroup of a colony
    ##returns a dataframe of a copy of bacteria object values (NOT OBJECTS)
    
    rows=[]
    for i in bac_list:
        rows.append(i.build_series())

    col_df = pd.DataFrame(rows)
    

    return col_df

In [21]:
##Creates a number of copies depending on state reproduction rate of a given bacteria object
##Returns a list of offspring bacteria objects

##Capable of being used in a .apply() manner

def reproduce(bac,sus_growth = 2,res_growth = 1.1,acq_growth = 1.1):
    ##Creates reproduced bacteria based on given bacteria
    
    offspring = []
    
    if bac.state == 0:
        rep_rate = sus_growth
        
    elif bac.state == 1:
        rep_rate = res_growth
        
    elif bac.state == 2:
        rep_rate = acq_growth
        
    else:
        raise Exception("Unknown Bacteria Object state")
            
            
    for i in range(round(rep_rate)):
        offspring.append(copy.copy(bac))
            
    
    return offspring

## Testing Zone

In [5]:
col = [bacteria(1,'streptococcus',1,'ArB1',0,0,'ocean'),bacteria(0,'sdfsa',1,'ArB1',0,0,'ocean'),bacteria(1,'streptococcus',1,'ArB1',0,0,'ocean')]
for i in range(3):
    col.append(bacteria(0,'dafsdva',0,0,0,0,'ocean'))
    
for i in range(3):
    col.append(bacteria(2,'dafsdva',0,0,0,0,'ocean'))
    
for i in range(5):
    col.append(bacteria(1,'dafsdva',0,0,0,0,'ocean'))

In [6]:
len(col)

14

In [7]:
len([x for x in col if x.state == 2])

3

In [8]:
horizontal_gene_transfer(col,res_trans_rate=.3,acq_trans_rate=.6)

3.9
[<__main__.bacteria object at 0x000001D84B3317F0>, <__main__.bacteria object at 0x000001D84B331860>, <__main__.bacteria object at 0x000001D84B3318D0>, <__main__.bacteria object at 0x000001D84B3311D0>]


In [9]:
len([x for x in col if x.state == 2])

7

In [12]:
group_df([x for x in col if x.state == 2])

,state,taxonomy,gene_pres,gene,mge_pres,mge,biome
0,2,sdfsa,1,ArB1,0,0,ocean
1,2,dafsdva,0,0,0,0,ocean
2,2,dafsdva,0,0,0,0,ocean
3,2,dafsdva,0,0,0,0,ocean
4,2,dafsdva,0,0,0,0,ocean
5,2,dafsdva,0,0,0,0,ocean
6,2,dafsdva,0,0,0,0,ocean


In [25]:
test = copy.copy(col[0])
test.state = 1
test.display()

State:  1
Taxonomy:  streptococcus
ARG Presence:  1
ARG:  ArB1
MGE Presence:  0
MGE:  0
Biome:  ocean



In [27]:
reproduce(test, res_growth=5)